In [3]:
from langchain import hub
from typing import Annotated, Literal, Sequence, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\active-learning\GENERATIVEAI\genai_bootcamp\nenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\active-learning\GENERATIVEAI\genai_bootcamp\nenv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Joejoyal\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run P

In [9]:
from langchain_groq import ChatGroq

In [18]:
llm = ChatGroq(model_name="gemma2-9b-it")

In [19]:
llm.invoke("Hi, How are you!")

AIMessage(content="As an AI, I don't have feelings, but I'm here and ready to help! How can I assist you today? 😊\n", response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 15, 'total_tokens': 47, 'completion_time': 0.058181818, 'prompt_time': 8.308e-05, 'queue_time': 0.019101200000000002, 'total_time': 0.058264898}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8c4f2c60-a7cd-416e-897f-80204a485209-0', usage_metadata={'input_tokens': 15, 'output_tokens': 32, 'total_tokens': 47})

In [36]:
urls = [
    "https://ageyetech.com/hyve/",
    "https://ageyetech.com/digitalcultivation/",
    "https://ageyetech.com/intelligence/"
]

In [37]:
docs = [WebBaseLoader(url).load() for url in urls]

In [38]:
docs

[[Document(metadata={'source': 'https://ageyetech.com/hyve/', 'title': 'HYVE - AGEYE Technologies', 'language': 'en-US'}, page_content='  HYVE - AGEYE Technologies                             SolutionsGrowOperationsIntelligenceCompanyAboutMissionPress and BlogsJobsRequest Demo AGEYE Grow A fully automated and scalable vertical farming system, integrating cutting-edge robotics, fertigation, and robust racking. What is HYVE? Hydroponic Year-Round Vertical Ecosystem A comprehensive, turnkey solution that merges modular racking systems with advanced robotic automation for effortless crop management and farm operations, empowering indoor growers to cultivate a broader range of crops with increased yields and significantly reduced operating costs. A Complete, Automated and Scalable System from One Source Full Automation Efficient and practical robotics technology significantly reduces human errors, improves plant health, and lowers labor costs, all without the substantial financial investmen

In [35]:
docs[0][0].metadata

{'source': 'https://ageyetech.com/intelligence/',
 'title': 'Intelligence - AGEYE Technologies',
 'language': 'en-US'}

In [40]:
docs[1][0].metadata

{'source': 'https://ageyetech.com/digitalcultivation/',
 'title': 'Digital Cultivation - AGEYE Technologies',
 'language': 'en-US'}

In [41]:
docs[2][0].metadata

{'source': 'https://ageyetech.com/intelligence/',
 'title': 'Intelligence - AGEYE Technologies',
 'language': 'en-US'}

In [46]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap=5)

In [44]:
docs_list = [item for sublist in docs for item in sublist]

In [45]:
docs_list

[Document(metadata={'source': 'https://ageyetech.com/hyve/', 'title': 'HYVE - AGEYE Technologies', 'language': 'en-US'}, page_content='  HYVE - AGEYE Technologies                             SolutionsGrowOperationsIntelligenceCompanyAboutMissionPress and BlogsJobsRequest Demo AGEYE Grow A fully automated and scalable vertical farming system, integrating cutting-edge robotics, fertigation, and robust racking. What is HYVE? Hydroponic Year-Round Vertical Ecosystem A comprehensive, turnkey solution that merges modular racking systems with advanced robotic automation for effortless crop management and farm operations, empowering indoor growers to cultivate a broader range of crops with increased yields and significantly reduced operating costs. A Complete, Automated and Scalable System from One Source Full Automation Efficient and practical robotics technology significantly reduces human errors, improves plant health, and lowers labor costs, all without the substantial financial investment

In [47]:
docs_splits = text_splitter.split_documents(docs_list)

In [48]:
docs_splits

[Document(metadata={'source': 'https://ageyetech.com/hyve/', 'title': 'HYVE - AGEYE Technologies', 'language': 'en-US'}, page_content='HYVE - AGEYE Technologies                             SolutionsGrowOperationsIntelligenceCompanyAboutMissionPress and BlogsJobsRequest Demo AGEYE Grow A fully automated and scalable vertical farming system, integrating cutting-edge robotics, fertigation, and robust racking. What is HYVE? Hydroponic Year-Round Vertical Ecosystem A comprehensive, turnkey'),
 Document(metadata={'source': 'https://ageyetech.com/hyve/', 'title': 'HYVE - AGEYE Technologies', 'language': 'en-US'}, page_content='A comprehensive, turnkey solution that merges modular racking systems with advanced robotic automation for effortless crop management and farm operations, empowering indoor growers to cultivate a broader range of crops with increased yields and significantly reduced operating costs. A Complete, Automated and Scalable System from One Source Full Automation Efficient and 

In [52]:
vectorstore = Chroma.from_documents(
    documents = docs_splits,
    collection_name = "rag-chroma",
    embedding = embeddings
)

In [54]:
vectorstore.as_retriever()

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000017CBFCD7D30>)

In [55]:
retriever=vectorstore.as_retriever()

In [56]:
type(retriever)

langchain_core.vectorstores.base.VectorStoreRetriever

In [57]:
retriever_tool = create_retriever_tool(
    retriever,
    "retriever_blog_posts",
    "Search and return information about ageye technologies blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.You are a specialized assistant. Use the 'retriever_tool' **only** when the query explicitly relates to LangChain blog data. For all other queries, respond directly without using any tool. For simple queries like 'hi', 'hello', or 'how are you', provide a normal response.",
)

In [58]:
tools=[retriever_tool]

In [59]:
tools

[Tool(name='retriever_blog_posts', description="Search and return information about ageye technologies blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.You are a specialized assistant. Use the 'retriever_tool' **only** when the query explicitly relates to LangChain blog data. For all other queries, respond directly without using any tool. For simple queries like 'hi', 'hello', or 'how are you', provide a normal response.", args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000017C8E0F70A0>, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000017CBFCD7D30>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000017C8E0F7130>, retriever=VectorStore

In [23]:
def AI_Assistant(state):
    pass

In [24]:
def retriever(state):
    pass

In [25]:
def rewriter(state):
    pass

In [26]:
def generate(state):
    pass

In [27]:
class AgentState:
    pass

In [28]:
workflow = StateGraph(AgentState)
workflow.add_node("ai assists", AI_Assistant)
workflow.add_node("retriever", retriever)
workflow.add_node("rewriter", rewriter)
workflow.add_node("generate", generate)

In [ ]:
workflow.add_edge()
workflow.add_conditional_edges()

In [ ]:
app = workflow.compile()